In [0]:
%tensorflow_version 1.x

In [0]:
import tensorflow as tf
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import PIL
from tensorflow.keras import layers
import time
import pickle

from IPython import display

In [0]:
#train_dataset = tf.data.Dataset.from_tensor_slices(mnist_train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

def training_batches():
  path = "C:\\Users\\abhin\\Downloads\\cifar-10-batches-py"
  file = "data_batch_1"
  #file = pd.concat(file,next)

  #file_path = os.path.join(path,file)

  with open(file, 'rb') as fo:
    training_batch = pickle.load(fo, encoding='bytes')  #retruns a dictionary
  
  features = training_batch[b'data']
  labels = training_batch[b'labels']
  return features, labels

In [0]:
cd gdrive/My \Drive

In [0]:
# N refers to the number of images in a batch, 
#H refers to the number of pixels in the vertical dimension, 
#W refers to the number of pixels in the horizontal dimension, 
#and C refers to the channels (e.g. 1 for black and white, 3 for RGB, etc.

#*************************************************************************#

#Some information about the dataset
#Data: 10000x3072 numpy array of uint8s. Each row stores a 32x32 color image
      #The first 1024 contains red channel values
      #Next 1024 contains green channel values
      #Final 1024 contains the blue channel values
      #The image is stored in row-major order, so that the first 32 entries
      #of the array are the red channel values of the first row of the image
#Labels: 10000 numbers in the range 0-9. The number at index i indicates the 
        # label of the ith image in the array data
#Label_names: A 10-element list which gives meaningful names to the numeric
            #labels in the labels array described above. For example 
            #label_names[0] == "airplane", label_name[1] == "automobile"

#*************************************************************************#

In [0]:
class DCGAN():
  def __init__(self):
    self.batch_size = 10000
    self.image_size = 32*32
    self.image_channels = 3
  
  def inputs(self):
    x = tf.placeholder(tf.float32, shape=(None, 32, 32, 3), name='input_x')
    z = tf.placeholder(tf.float32, shape=(None, 32, 32, 3), name='noise')
    prob = tf.placeholder(tf.float32, name='prob')

    return x, z, prob

  def dcgan_generator_model(self, input, reuse = False):
    with tf.variable_scope("dcgan_generator_model") as scope:
      if reuse:
        scope.reuse_variables()
   
    #tf.nn.conv2d_transpose(input, filters, output_shape, strides, padding='SAME', data_format='NHWC',dilations=None, name=None)
    #First Convolution Layer
    g_conv_layer1 = tf.nn.conv2d_transpose(input, [None, 32, 32, 3], [1,2,2,1]) #Added by Abhinav 
    g_conv_layer1 = tf.nn.relu(tf.contrib.layers.batch_norm(g_conv_layer1,decay = 0.9, epsilon=1e-5))

    #Second Convolution Layer
    g_conv_layer2 = tf.nn.conv2d_transpose(g_conv_layer1, [None, 32, 32, 3], [1,2,2,1])
    g_conv_layer2 = tf.nn.relu(tf.contrib.layers.batch_norm(g_conv_layer2,decay = 0.9, epsilon=1e-5))

    #Thrid Convolution Layer
    g_conv_layer3 = tf.nn.conv2d_transpose(g_conv_layer2, [None, 32, 32, 3], [1,2,2,1])
    g_conv_layer3 = tf.nn.relu(tf.contrib.layers.batch_norm(g_conv_layer3,decay = 0.9, epsilon=1e-5))

    #Fourth Convolution Layer
    g_conv_layer4 = tf.nn.conv2d_transpose(g_conv_layer3, [None, 32, 32, 3], [1,2,2,1])
    g_conv_layer4 = tf.nn.relu(tf.contrib.layers.batch_norm(g_conv_layer4,decay = 0.9, epsilon=1e-5))

    output = tf.nn.tanh(g_conv_layer4) #if the dataset is the MNIST

    return output

  def dcgan_discriminator_model(self, input, reuse = False):

    with tf.variable_scope("discriminator") as scope:
      if reuse:
        scope.reuse_variables()
    
    #First Convoultion Layer
    d_conv1 = tf.nn.conv2d(input, strides = [1, 2, 2, 1], padding='SAME')
    d_conv1 = tf.nn.leaky_relu(d_conv1, alpha = 0.2)

    #Second Convolution Layer
    d_conv2 = tf.nn.conv2d(d_conv1, strides = [1, 2, 2, 1], padding='SAME')
    d_conv2 = tf.nn.leaky_relu(tf.contrib.layers.batch_norm(d_conv2, decay=0.9, epsilon=1e-5), alpha = 0.2)

    #Third Convolution Layer
    d_conv3 = tf.nn.conv2d(d_conv2, strides = [1, 2, 2, 1], padding='SAME')
    d_conv3 = tf.nn.leaky_relu(tf.contrib.layers.batch_norm(d_conv3, decay=0.9, epsilon=1e-5), alpha = 0.2)

    #Fourth Convolution Layer
    d_conv4 = tf.nn.conv2d(d_conv3, strides = [1, 2, 2, 1], padding='SAME')
    d_conv4 = tf.nn.leaky_relu(tf.contrib.layers.batch_norm(d_conv4, decay=0.9, epsilon=1e-5), alpha = 0.2)

    hidden = tf.reshape(d_conv4, [10000, 32, 32, 3])

    output = tf.nn.sigmoid(tf.matmul(hidden, D_FW1) + D_Fb1)

    return output

  def loss(self, x, z):
    g_output = self.dcgan_generator_model(z)
    d_fake = self.dcgan_discriminator_model(g_output, reuse= False)
    d_real = self.dcgan_discriminator_model(x, reuse = True)

    d_loss = tf.reduce_mean(tf.log(d_real)+tf.log(1 - d_fake))
    g_loss = tf.reduce_mean(tf.log(d_fake))

    return d_loss, g_loss

  def optimizer(self, d_loss, g_loss):
    d_var_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='discriminator')
    g_var_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='generator')

    d_opt = tf.train.AdamOptimizer(0.0002, beta1 = 0.5).minimize(-d_loss,
                                                            var_list=d_var_list)
    g_opt = tf.train.AdamOptimizer(0.0002, beta1 = 0.5).minimize(-g_loss,
                                                            var_list=g_var_list)
    return d_opt, g_opt

  def sample(self, Z, reuse = True):
    g_out=self.generator(Z, reuse=reuse)
    return g_out

In [129]:
if __name__ == "__main__":
  #getting the training batch
  
  features, labels = training_batches()

  input_x = tf.reshape(features, [10000, 32, 32, 3])
  dcgan = DCGAN()
  x = tf.placeholder(tf.float32, shape=(None, 32, 32, 3), name='input_x')
  z = tf.placeholder(tf.float32, shape=(None, 32, 32, 3), name='noise')
  #x, z, probability = dcgan.inputs()
  print(x.shape)
  disc_loss, gen_loss = dcgan.loss(x, z)
  disc_opt, gen_opt = dcgan.optimizer(disc_loss, g_loss)

  init = tf.global_variables_initializer()

  sess = tf.Session()
  sess.run(init)
  writer = tf.summary.FileWriter("./log", sess.graph)

  for epoch in range(epochs):

    #Generator noise to feed the discriminator
    z_noise = np.random.uniform(-1.,1., size = [10000, 3072])
    for i in range(5):
      _, Disc_loss_epoch = sess.run([disc_opt, disc_loss], feed_dict={x:input_x, z:z_noise})

    _, Gen_loss_epoch = sess.run([gen_opt, gen_loss], feed_dict = {z_input:z_noise})

    #running the Discriminatory summary
    #summary_Disc_Loss = sess.run(Disc_loss_total, feed_dict = (x_input:x_batch,z_input:z_noise))
    #Adding the Generator summary
    writer.add_summary(Disc_loss_epoch, epoch)

    #Running the Generator summary
    #summary_Gen_Loss = sess.run(Gen_loss_total, feed_dict = (z_input:z_noise))
    #Adding the Generator summary
    writer.add_summary(Gen_loss_epoch, epoch)

    if epoch % 2000 == 0:
      print("steps: {0} : Generator_loss:{1} : Discriminator_loss:{2}".format(epoch,Gen_loss_epoch,Disc_loss_epoch))


    #Testing 
    #Generate images from noise, using the generator network
    n=6
    canvas = np.empty((28*n, 28*n))
    for i in range(n):
      #Noise input
      z_noise = tf.random.uniform(-1.,1., size=[batch_size, z_noise_dim])
      #Generate image from noise
      g=sess.run(output_Gen, feed_dict={z_input:z_noise})
      #Reverse colors for better display
      g = -1 * (g-1)
      for j in range(n):
        #Draw the generated digits
        canvas[i * 28:(i+1)*28, j * 28:(j+1)*28] = g[j].reshape([28,28])
  
    plt.figure(figsize=(n,n))
    plt.imshow(canvas, origin="upper", cmap="gray")
    plt.show()

(?, 32, 32, 3)


TypeError: ignored